In [1]:
import autosklearn.classification
import csv
import json
import math
import numpy as np
import re
import warnings

from collections import defaultdict, OrderedDict
from myconfig import *
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import StratifiedShuffleSplit

N_GRAM_LENGTH = 10
TOTAL_TYPE = 2
summary = dict()
total_n_gram = 0
total_document = 0
score = dict()
id_n_gram_mapping = dict()
comment_summary = dict()
top_vector = list()
n_grams = dict()
## Change project here
project_name = "appreviews"
print("stratified shuffle split "+project_name)

# negative = -1, neutral = 0, postitive = 1
# weight = log(|D|/sdf) * gtf

def read_raw_data():
    with open(DATA_SOURCE+"/dataset.csv", 'r',encoding='latin1') as csvfile:
        global total_document
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['project'] == project_name:
                comment_summary[total_document] = dict()
                comment_summary[total_document]['type'] = int(row['oracle'])
                comment_summary[total_document]['comment'] = row['text']
                total_document += 1

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1   
    
#read data from n_gram
def read_n_gram():
    n_gram_id = 0
    total_n_gram = file_len(DATA_SOURCE+"/"+project_name+"_n_gram_filter")
    print("total_n_gram",total_n_gram)
    with open(DATA_SOURCE+"/"+project_name+"_n_gram_filter") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in reader:
            words = row[5].strip()
            term = tuple(row[5].strip().split(' '))
            if term not in summary:
                summary[term] = dict()
            summary[term] = {'id':n_gram_id,'len':row[1],'gtf':row[2],'df':row[3],'sdf':row[4], 'term':row[5]}
            weight1 = math.log10(total_document / int(row[4]))
            summary[term]['score'] = weight1 * int(row[2])
            score[n_gram_id] =  weight1 * int(row[2])
#            weight2 = math.log10(total_document * int(row[3]) / (int(row[4]) * int(row[4])) )
#            summary[term]['score'] = weight2
#            score[n_gram_id] =  weight2
            n_grams[n_gram_id] = term
            n_gram_id += 1

def top_score_vector():
    percent = int(len(score) * 100 / 100)
    top_vector.extend(sorted(score,key=score.get,reverse=True)[:percent])
#     top_vector.extend(list(filter(lambda w: score[w] > 1, score)))
    print("top_vector",len(top_vector))

def n_gram_split():
    for comment_index in comment_summary:
        comment = comment_summary[comment_index]['comment']
        comment_summary[comment_index]['vector'] = dict()
        comment_post_process = re.sub("\s+"," ",re.sub(r"[^A-Za-z0-9]+"," ",comment.replace("\t"," ").replace("\r\n"," ").lower())).split(" ")
        for i in range(len(comment_post_process)):
            for j in range(i,min(i+N_GRAM_LENGTH+1,len(comment_post_process))):
                if(tuple(comment_post_process[i:j+1]) in summary):
                    if(summary[tuple(comment_post_process[i:j+1])]['id'] in top_vector):
                        if summary[tuple(comment_post_process[i:j+1])]['id'] in comment_summary[comment_index]['vector']:
                            comment_summary[comment_index]['vector'][summary[tuple(comment_post_process[i:j+1])]['id']] += 1
                        else: 
                            comment_summary[comment_index]['vector'][summary[tuple(comment_post_process[i:j+1])]['id']] = 1
                            
def vector_idf():
    for i in comment_summary:
         for v in comment_summary[i]['vector']:
            comment_summary[i]['vector'][v] *= score[v]

def cal_coverage():
    empty_vector = 0
    total_vector = len(comment_summary)
    for i in comment_summary:
        if not comment_summary[i]['vector']:
            empty_vector += 1
            print(comment_summary[i],comment_summary[i]['vector'])
    print("coverage ratio:",(total_vector - empty_vector)/total_vector * 100)

def cal_coverage_freq():
    empty_vector = 0
    total_vector = len(comment_summary)
    n_gram_per_comment = defaultdict(lambda:0)
    for i in comment_summary:
        sum = 0
        for j in comment_summary[i]['vector']:
            sum += 1
        n_gram_per_comment[sum] += 1
    print(OrderedDict(sorted(n_gram_per_comment.items(), key=lambda t: t[0])))
    
read_raw_data()
read_n_gram()
top_score_vector()
n_gram_split()
print("finish")
#vector_idf()
cal_coverage()
cal_coverage_freq()

/Users/rungrojmaipradit/anaconda3/lib/python3.7/site-packages/pyparsing.py:2725: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )


stackoverflow
total_n_gram 3050
top_vector 3050
finish
{'type': 0, 'comment': 'Gotcha ', 'vector': {}} {}
{'type': 0, 'comment': 'Trust URI ', 'vector': {}} {}
{'type': 0, 'comment': 'InsertandRetriveBlobData ', 'vector': {}} {}
{'type': 0, 'comment': 'Hmmm ', 'vector': {}} {}
{'type': 1, 'comment': 'Worked ', 'vector': {}} {}
{'type': 1, 'comment': 'Congratulations ', 'vector': {}} {}
coverage ratio: 99.6
OrderedDict([(0, 6), (1, 16), (2, 33), (3, 45), (4, 45), (5, 62), (6, 53), (7, 69), (8, 72), (9, 70), (10, 72), (11, 76), (12, 79), (13, 82), (14, 62), (15, 63), (16, 42), (17, 70), (18, 44), (19, 32), (20, 37), (21, 44), (22, 34), (23, 39), (24, 35), (25, 33), (26, 23), (27, 16), (28, 18), (29, 19), (30, 11), (31, 15), (32, 9), (33, 8), (34, 11), (35, 9), (36, 5), (37, 4), (38, 6), (39, 3), (40, 4), (41, 4), (42, 2), (44, 3), (45, 1), (46, 2), (47, 1), (49, 4), (50, 1), (52, 1), (53, 2), (55, 2), (59, 1)])


In [2]:
# currently we have top_score_vector and comment&vector
# create x and y -> x contains vector and y contains result
X = []
y = []
print("test")
for comment_index in comment_summary:
    vector = [0] * len(top_vector)
    for vector_index in comment_summary[comment_index]['vector']:
        vector[top_vector.index(vector_index)] = comment_summary[comment_index]['vector'][vector_index]
    X.append(vector)
    y.append(comment_summary[comment_index]['type'])
print("finish")

test
finish


In [3]:
warnings.filterwarnings("ignore")

sss = StratifiedShuffleSplit(n_splits=10,random_state=1)
np_X,np_y = np.asarray(X),np.asarray(y)
sss.get_n_splits(np_X, np_y)
runner = 0

measure_scores = dict()
# measure_scores['precision'] = list()
# measure_scores['recall'] = list()
# measure_scores['f1'] = list()
measure_scores['AUC'] = list()
for train_index, test_index in sss.split(np_X,np_y):
    X_train, X_test = np_X[train_index], np_X[test_index]
    y_train, y_test = np_y[train_index], np_y[test_index]
    automl = autosklearn.classification.AutoSklearnClassifier(ml_memory_limit=1024*64,time_left_for_this_task=90*60,per_run_time_limit=15*60)
    automl.fit(X_train.copy(), y_train.copy(),metric=autosklearn.metrics.f1_weighted)
    automl.refit(X_train.copy(), y_train.copy())
    y_hat = automl.predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_hat, pos_label=1)
    measure_scores['AUC'].append(auc(fpr, tpr))
    log = automl.cv_results_
    with open(project_name+"_sss_"+str(runner)+".txt","w") as file:
        file.write("log for "+str(runner)+"\n")
        file.write(str(log)+"\n")
    
    print("round:",runner,"Classification report", classification_report(y_test, y_hat))
    print("round:",runner,"Confusion matrix", confusion_matrix(y_test, y_hat))
    print("round:",runner,"AUC",auc(fpr,tpr))
    print("number of process",len(log['rank_test_scores']))
    print("status",log['status'])
    print("mean_test_score",log['mean_test_score'])
    
    for i in np.where(log['rank_test_scores']==1)[0] :
        print(log['params'][i])     

    for i in range(len(y_test)):
        if y_test[i] != y_hat[i]:
            print("round:",runner,"comment index:",test_index[i],"y_test:",y_test[i],"y_hat",y_hat[i])
        
    runner += 1

print('AUC',measure_scores['AUC'],np.mean(measure_scores['AUC']))

[WARNING] [2019-01-15 14:23:50,015:EnsembleBuilder(1):549cc566db55aa91e299a80b32babd66] No models better than random - using Dummy Score!
[WARNING] [2019-01-15 14:23:50,030:EnsembleBuilder(1):549cc566db55aa91e299a80b32babd66] No models better than random - using Dummy Score!
[WARNING] [2019-01-15 14:23:52,036:EnsembleBuilder(1):549cc566db55aa91e299a80b32babd66] No models better than random - using Dummy Score!
[WARNING] [2019-01-15 14:24:02,063:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-01-15 14:24:02,063:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[14:25:56] src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[14:25:56] src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[14:25:56] src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[14:25:56] src/gbm/gbtree.cc:492: drop 0 trees, weight = 1
[14:25:56] src/gbm/gbtree.cc:492: drop 0